In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from preprocessing import preprocessing
import matplotlib.pyplot as pyplot
pyplot.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

from xgboost import XGBRegressor

### Funciones Auxiliares

In [3]:
def leer_csv(ruta):
    return pd.read_csv(ruta)

In [4]:
def separar_train_test(df_train, logaritmico):
    X = df_train.copy().drop(columns = ['precio', 'log_precio'])
    if logaritmico:
        y = df_train['log_precio']
    else:
        y = df_train['precio']
    return train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)
    

In [5]:
def mae_exp(x, y):
    return mean_absolute_error(np.exp(x), np.exp(y))

## Función de entrenamiento para los modelos

In [6]:
def entrenar(modelo, ruta_df_train, nombre_modelo, logaritmico):

    df_train = leer_csv(ruta_df_train)
    
    X_train, X_test, Y_train, Y_test = separar_train_test(df_train, logaritmico)
    
    resultado = []
    

    print('------------------------------------------------')
    print('Comienza a entrenar: '+ nombre_modelo)
    print()

    modelo = modelo   
    modelo.fit(X_train, Y_train)
    prediccion = modelo.predict(X_test)

    if logaritmico:
        score = mean_absolute_error(np.exp(prediccion), np.exp(Y_test))
    else:
        score = mean_absolute_error(prediccion, Y_test)

    resultado.append((str(nombre_modelo), str(score)))

    print('**FINALIZADO** ' + nombre_modelo + ' Score [MAE]: ' + str(score))
    print()
    return resultado, modelo



In [11]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

RUTA_TRAIN_CON_NULLS = 'data/train_preproc_con_nulls.csv'
RUTA_TRAIN_SIN_NULLS = 'data/train_preproc_sin_nulls.csv'
RUTA_TEST_CON_NULLS = 'data/test_preproc_con_nulls.csv'
RUTA_TEST_SIN_NULLS = 'data/test_preproc_sin_nulls.csv'

CON_PRECIO_LOGARITMICO = True

## XGBoost

In [12]:
modelo_xgboost = XGBRegressor(n_estimators=150,max_depth=21,min_child_weight =2,eta = 0.1,\
                              subsample=0.9,colsample_bytree = 0.8,n_jobs = 8, random_state=RANDOM_STATE)
nombre_xgboost = 'XGBoost'

In [15]:
resultado, modelo_xgboost =  entrenar(modelo_xgboost, RUTA_TRAIN_CON_NULLS, nombre_xgboost, CON_PRECIO_LOGARITMICO)

------------------------------------------------
Comienza a entrenar: XGBoost

[19:30:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
**FINALIZADO** XGBoost Score [MAE]: 487062.8823437501



## RandomForest Regressor

In [10]:
modelo_random_forest = RandomForestRegressor(n_estimators = 200, max_depth = 500, n_jobs = -1)
nombre_random_forest = 'RandomForestRegressor'

resultado, modelo_random_forest =  entrenar(modelo_random_forest, RUTA_TRAIN_SIN_NULLS, nombre_random_forest, CON_PRECIO_LOGARITMICO)

#500683.88162094826

------------------------------------------------
Comienza a entrenar: RandomForestRegressor

**FINALIZADO** RandomForestRegressor Score [MAE]: 499026.67681519437



## Promedio de las predicciones

In [22]:
___, X_test_sin_nulls, __, Y_test = separar_train_test(leer_csv(RUTA_TRAIN_SIN_NULLS), True)
predicciones_random_forest = modelo_random_forest.predict(X_test_sin_nulls)

___,X_test_con_nulls,_,__ = separar_train_test(leer_csv(RUTA_TRAIN_CON_NULLS), True)
predicciones_xgboost = modelo_xgboost.predict(X_test_con_nulls)

mean_absolute_error(np.exp((predicciones_xgboost+predicciones_random_forest)/2), np.exp(Y_test))

406769.1317725219

_______________________
## Función para entrega en kaggle

In [13]:
def predecir(modelo, ruta_train, ruta_test):
    
    df_train = leer_csv(ruta_train).drop(columns = ['precio', 'log_precio'])
    to_predict = leer_csv(ruta_test)
    
    columnas_train = df_train.columns
    to_predict = to_predict.reindex(columns = columnas_train)
    
    predicciones = modelo.predict(to_predict)
    
    return predicciones

In [16]:
predicciones_modelo_xgboost = predecir(modelo_xgboost, RUTA_TRAIN_CON_NULLS, RUTA_TEST_CON_NULLS)
predicciones_modelo_random_forest = predecir(modelo_random_forest, RUTA_TRAIN_SIN_NULLS, RUTA_TEST_SIN_NULLS)

In [17]:
def crear_df_predicciones(lista_predicciones):
    
    df_test_original = leer_csv('data/test.csv')
    Id = df_test_original['id']
    
    df_resultado = pd.DataFrame()
    
    df_resultado['id'] = Id
    
    prediccion = np.exp(lista_predicciones[0])
    for i in range(1, len(lista_predicciones)):
        prediccion += np.exp(lista_predicciones[i])
    
    df_resultado['target'] = prediccion/len(lista_predicciones)
    
    return df_resultado

In [23]:
df_predicciones = crear_df_predicciones([predicciones_modelo_random_forest, predicciones_modelo_xgboost])

In [25]:
df_predicciones.to_csv('data/prediccion.csv', index=False)

___________
# Resultados guardados

**FINALIZADO** XGBoost Score: 543519.3351068652

**FINALIZADO** XGBoost Score: 543808.3471162983 //Antes de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 500191.9174657849 //Despues de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 501766.49058353336 //Despues de agregar gimnasio, usos multiples y piscina

**FINALIZADO** XGBoost Score: 500862.1657935754 // Agrego solo piscina.

**FINALIZADO** XGBoost Score: 499181.4921723928 //Agrego precio promedio por m2 por provincia. Dejando piscina mejora.

**FINALIZADO** XGBoost Score: 496056.96402535716 //Tony agrega lat+lng precio promedio por m2

**FINALIZADO** XGBoost Score: 495398.1192696873 //Agrego feature tiene_sup_descubierta

**FINALIZADO** XGBoost Score: 491674.73406999116 //Agrego feature diff_metros_totales y metros_cubiertos

**FINALIZADO** XGBoost Score: 491335.880882687 //Agrego one hot encoding para antiguedad

**FINALIZADO** XGBoost Score: 490347.0285999908 //Dejando columna antiguedad Y el one hot encoding (antes se dropeaba)

**FINALIZADO** XGBoost Score: 489407.5245960651 //No imputando. Poniendo listas vacias

**FINALIZADO** XGBoost Score: 487797.2694374844 // **ENTREGA** USANDO 0.1 DE TEST_SIZE

**FINALIZADO** XGBoost Score:485399.4606591214 // con palabras [increíble excelente hermos espectacular exclusivo] para palabras importantes

**FINALIZADO** XGBoost Score: 482156.3098664535 //con palabras ['cuarto servicio','para auto','baño completo','sala comedor','cocina integral','baño vestidor'] para palabras importantes

**FINALIZADO** XGBoost Score: 480918.3753545355 //con palabras ['cuarto servicio','para auto','baño completo','baño vestidor','exclusivo'] para palabras importantes

**FINALIZADO** XGBoost Score: 475666.92711221 // **Entrege** con 0.05 DE TEST_SIZE con palabras ['roof garden', 'doble altura', 'sala tv', \
           'lugares estacionamiento','cajones estacionamiento',\
            'family room','double altura','salón juego','ideal para',\
            'amplios espacio', 'independiente', 'casa condominio',\
            'salón juego', 'pisos madera','exclusivo'] para palabras importantes

--------------------------------------------------------------------

**FINALIZADO** XGBoost Score [MAE]: 486560.74267065345 // One hot encoding de habitaciones y banos






**FINALIZADO** RandomForestRegressor Score [MAE]: 500686.94137633865 // One hot encoding banos y habitaciones